In [1]:
from langchain_groq import ChatGroq
import os
#from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceBgeEmbeddings
#from langchain.vectorstores import Pinecone as PCN
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain import HuggingFaceHub
#from sentence_transformers import SentenceTransformer
#from huggingface_hub import HuggingFaceHub
from langchain.llms import CTransformers

In [2]:
GROG_API_KEY = os.environ.get("GROG_API_KEY")
PINECONE_KEY = os.environ.get("PINECONE_KEY")
HUGGINGFACEHUB_API_TOKEN = os.environ.get("HUGGINGFACEHUB_API_TOKEN")

In [3]:
PINECONE_KEY

'pcsk_dH9R3_PNhMeN3NWVGZ3GoN3XWrASbMdMLctr4qtwFGN37QzYoUAnCygmb6d6FoeXq3huP'

In [3]:
llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0.5,
    api_key=GROG_API_KEY,
    max_tokens=4096,
    max_retries=2
)

In [ ]:
# Extract the PDF
#def document_loader(data):
    #loader = DirectoryLoader(data,
                    #glob="*.pdf",
                    #loader_cls=PyPDFLoader)
    #documents = loader.load()
    #return documents

In [ ]:
#extracted_data = document_loader("../data/")

In [ ]:
# Create or split text chunks
# def text_split(extracted_data):
#     text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500,
#                                    chunk_overlap = 20)
#     text_chunks = text_splitter.split_documents(extracted_data)
#     return text_chunks

In [ ]:
# text_chunks = text_split(extracted_data=extracted_data)

In [ ]:
#text_chunks

In [4]:
# Download Embedding Model
#local_model_path = "../model/"
def download_hugging_face_embeddings(modelname):
    embeddings = HuggingFaceBgeEmbeddings(model_name=modelname,)
    return embeddings

In [5]:
embeddings = download_hugging_face_embeddings("sentence-transformers/all-MiniLM-L6-v2")

d:\LLMProject\Medical_Chat_Bot\mbot\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# query_result = embeddings.embed_query("Hello World")
# print(f"Length : {len(query_result)}")

Length : 384


In [ ]:
#from pinecone import Pinecone
#pc = Pinecone(api_key=PINECONE_KEY)
#pc_index = pc.Index("medical-chatbot")

In [ ]:
#docsearch = Pinecone.from_texts([t.page_content for t in text_chunks],embedding=embeddings,index_name=pc_index)
#for i, t in zip(range(len(text_chunks)), text_chunks):
   #query_result = embeddings.embed_query(t.page_content)
   #pc_index.upsert(
   #vectors=[
        #{
            #"id": str(i),  # Convert i to a string
            #"values": query_result, 
            #"metadata": {"text":str(text_chunks[i].page_content)} # meta data as dic
        #}
    #],
    #namespace="real"
#)

In [6]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter
loader = PyPDFLoader("../data/Medical_book.pdf")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=20)
docs = text_splitter.split_documents(documents)

In [ ]:
#from langchain_community.document_loaders import TextLoader
#txt_loader = TextLoader("../data/payment_text.txt")
#txt_documents = txt_loader.load()

#txt_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
#text_docs = txt_splitter.split_documents(txt_documents)

In [ ]:
#text_docs

In [ ]:
#from langchain_pinecone import PineconeVectorStore

#os.environ['PINECONE_API_KEY'] = PINECONE_KEY

# index_name = "medical-chatbot"

# vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)

In [ ]:
# index_obj = vectorstore.get_pinecone_index(index_name)
# index_obj

In [ ]:
# vectorstore_from_docs = vectorstore.from_documents(
#     docs,
#     index_name=index_name,
#     embedding=embeddings
# )
# vectorstore_from_docs

In [ ]:
## Delete record from Pincecone
#from pinecone import Pinecone
#pc = Pinecone(api_key=PINECONE_KEY)
#index = pc.Index(host='https://medical-chatbot-a0ji3yb.svc.aped-4627-b74a.pinecone.io')
#index.delete(delete_all=True,namespace='default')

In [7]:
import chromadb

In [8]:
client = chromadb.PersistentClient('../chromadb/medicalstore')
collection = client.get_or_create_collection("medicalbook")

In [ ]:
#query = "What are Allergies"
#result =vectorstore.similarity_search(query=query,k=3)
#print(result)

[Document(id='74b79ad3-433b-4030-9c04-0ca6f46bfc44', metadata={'page': 128.0, 'source': '../data/Medical_book.pdf'}, page_content='Description\nAllergies are among the most common of medical\ndisorders. It is estimated that 60 million Americans, or\nmore than one in every five people, suffer from some\nform of allergy, with similar proportions throughout\nmuch of the rest of the world. Allergy is the single largest\nreason for school absence and is a major source of lost\nproductivity in the workplace.\nAn allergy is a type of immune reaction. Normally,\nthe immune system responds to foreign microorganisms\nor particles by producing specific proteins called anti-\nbodies. These antibodies are capable of binding to iden-\ntifying molecules, or antigens, on the foreign particle.\nThis reaction between antibody and antigen sets off a\nseries of chemical reactions designed to protect the\nbody from infection. Sometimes, this same series of\nreactions is triggered by harmless, everyday subs

In [10]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [11]:
PROMPT = PromptTemplate(template=prompt_template,input_variables=['context','question'])
chain_type_kwargs = {'prompt':PROMPT}

In [12]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={'k':2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])